# Model Tracking, logging, versioning and registry

In [27]:
import mlflow
from dotenv import load_dotenv
import os

load_dotenv()

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

## Notebook Naming Conventions

`0.01` – Keeps work in **chronological order**.  
The structure follows:  
**PHASE.NOTEBOOK-INITIALS-DESCRIPTION.ipynb**

### Structure Details
- **PHASE** — Indicates the project phase.  
- **NOTEBOOK** — Sequential number of the notebook within that phase.  
- **INITIALS** — Your initials (helps identify authors and avoid naming conflicts).  
- **DESCRIPTION** — Short description of what the notebook covers.

### Recommended Phases
You can adapt these to your workflow, but a common scheme is:

| Phase | Description | Typical Output |
|--------|--------------|----------------|
| **0 – Data Exploration** | Initial exploratory work, inspecting raw data. | EDA notes, quick insights. |
| **1 – Data Cleaning & Feature Creation** | Data preprocessing and feature engineering. | Cleaned datasets in `data/processed` or `data/interim`. |
| **2 – Visualizations** | Creating exploratory or publication-quality visualizations. | Figures for reports or papers. |
| **3 – Modeling** | Training, tuning, and evaluating machine learning models. | Trained models, metrics, serialized outputs. |
| **4 – Publication / Reporting** | Notebooks converted into final reports or presentations. | Markdown, PDFs, or published visuals. |

### Naming Components
- `0.01` → Phase 0, first notebook in that phase.  
- `ira` → Your initials.  
- `data-source-1` → Short description of the notebook’s focus.

# Autolog

In [28]:
mlflow.set_experiment("autolog-example")

2025/10/04 22:56:24 INFO mlflow.tracking.fluent: Experiment with name 'autolog-example' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:C:/Users/ivanr/apps/MLOps/mna-2.2-tracking/mlruns/1', creation_time=1759640184856, experiment_id='1', last_update_time=1759640184856, lifecycle_stage='active', name='autolog-example', tags={}>

In [29]:
import pandas as pd

df = pd.read_csv("../data/raw/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=["quality"])
y = df["quality"]

mlflow.sklearn.autolog()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
mlflow.start_run(run_name="random-forest-classifier")
model = RandomForestClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.6625

In [32]:
mlflow.end_run()

🏃 View run random-forest-classifier at: http://localhost:5000/#/experiments/1/runs/5d3412109d344fffb43f1ebbc83b9623
🧪 View experiment at: http://localhost:5000/#/experiments/1


# Loading Models

In [33]:
runs = mlflow.search_runs(
    search_all_experiments=True,
    order_by=["start_time DESC"],
    filter_string="status='FINISHED'"
)

In [34]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_score,metrics.training_accuracy_score,metrics.RandomForestClassifier_score_X_test,metrics.training_recall_score,...,params.random_state,params.ccp_alpha,params.min_weight_fraction_leaf,params.n_estimators,tags.mlflow.user,tags.estimator_name,tags.estimator_class,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName
0,5d3412109d344fffb43f1ebbc83b9623,1,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 04:58:18.005000+00:00,2025-10-05 04:59:23.491000+00:00,1.0,1.0,0.6625,1.0,...,None,0.0,0.0,100,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,random-forest-classifier


In [35]:
run_id = runs.run_id[0]
model_uri = f"runs:/{run_id}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [36]:
loaded_model.predict(X_test)

array([5, 5, 5, 5, 6, 5, 5, 5, 6, 6, 7, 5, 6, 5, 6, 7, 5, 6, 7, 5, 5, 6,
       5, 6, 5, 6, 6, 5, 5, 6, 5, 5, 6, 6, 6, 5, 6, 6, 5, 6, 5, 5, 6, 5,
       6, 6, 7, 6, 5, 6, 5, 5, 6, 7, 5, 6, 6, 6, 6, 5, 6, 6, 6, 5, 7, 6,
       7, 6, 7, 5, 6, 5, 6, 6, 6, 5, 7, 5, 6, 7, 5, 7, 5, 6, 6, 6, 5, 6,
       6, 5, 6, 5, 5, 5, 5, 5, 5, 6, 5, 6, 5, 5, 6, 7, 6, 7, 6, 5, 6, 5,
       7, 5, 7, 5, 6, 6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 5, 6, 5, 5, 6, 6, 5,
       5, 6, 6, 5, 5, 5, 5, 6, 7, 6, 7, 5, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5,
       6, 5, 6, 6, 5, 6, 5, 6, 6, 5, 5, 6, 6, 5, 5, 5, 5, 5, 7, 5, 7, 6,
       6, 5, 5, 5, 5, 6, 5, 6, 5, 7, 6, 6, 7, 5, 6, 6, 5, 6, 6, 5, 5, 6,
       5, 7, 5, 5, 5, 5, 7, 6, 5, 6, 6, 6, 8, 5, 5, 6, 6, 6, 6, 5, 6, 6,
       5, 6, 6, 6, 6, 5, 5, 7, 5, 5, 5, 5, 6, 6, 5, 7, 5, 6, 6, 5, 5, 5,
       6, 7, 5, 7, 6, 6, 6, 5, 6, 5, 5, 6, 6, 5, 6, 6, 6, 6, 6, 6, 5, 7,
       6, 6, 5, 5, 6, 6, 5, 6, 5, 6, 6, 6, 6, 6, 6, 6, 7, 5, 5, 5, 5, 7,
       5, 6, 5, 6, 5, 7, 6, 5, 5, 6, 5, 7, 6, 6, 5,

# Custom Logging and Nested Runs

In [37]:
mlflow.autolog(disable=True)

In [38]:
mlflow.set_experiment("rf-classifier-nested-tuning") 

param_grid = [
    {"n_estimators": 100, "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 1},
    {"n_estimators": 200, "max_depth": 8,    "min_samples_split": 2, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 12,   "min_samples_split": 5, "min_samples_leaf": 2},
    {"n_estimators": 400, "max_depth": None, "min_samples_split": 10,"min_samples_leaf": 1},
    {"n_estimators": 200, "max_depth": 16,   "min_samples_split": 2, "min_samples_leaf": 4},
]

2025/10/04 23:03:15 INFO mlflow.tracking.fluent: Experiment with name 'rf-classifier-nested-tuning' does not exist. Creating a new experiment.


In [39]:
from sklearn.metrics import accuracy_score

with mlflow.start_run(run_name="random-forest-classifier") as parent_run:
    mlflow.set_tag("stage", "tuning")
    mlflow.set_tag("model_family", "RandomForestClassifier")

    for i, params in enumerate(param_grid, 1):
        with mlflow.start_run(run_name=f"trial-{i}", nested=True):
            # train
            model = RandomForestClassifier(
                n_estimators=params["n_estimators"],
                max_depth=params["max_depth"],
                min_samples_split=params["min_samples_split"],
                min_samples_leaf=params["min_samples_leaf"],
                n_jobs=-1,
                random_state=42,
            ).fit(X_train, y_train)

            # eval
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            # log params/metrics + model
            mlflow.log_params(params)
            mlflow.log_metric("accuracy", acc)
            mlflow.sklearn.log_model(model, artifact_path="model")

2025/10/04 23:06:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 23:06:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trial-1 at: http://localhost:5000/#/experiments/2/runs/3827d61f38614c779acd5ad76bb250c1
🧪 View experiment at: http://localhost:5000/#/experiments/2


2025/10/04 23:06:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 23:06:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trial-2 at: http://localhost:5000/#/experiments/2/runs/f0ecb842d0cd461dac00f014a5c28aec
🧪 View experiment at: http://localhost:5000/#/experiments/2


2025/10/04 23:06:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 23:06:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trial-3 at: http://localhost:5000/#/experiments/2/runs/baf47c08bac542d59c0556288300f040
🧪 View experiment at: http://localhost:5000/#/experiments/2


c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlops-env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2025/10/04 23:06:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 23:06:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trial-4 at: http://localhost:5000/#/experiments/2/runs/20c57124c48045cfacde7123836d2d56
🧪 View experiment at: http://localhost:5000/#/experiments/2


c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlops-env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
2025/10/04 23:07:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/04 23:07:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trial-5 at: http://localhost:5000/#/experiments/2/runs/1281a19ad5da488dafd672f3ae0a2e83
🧪 View experiment at: http://localhost:5000/#/experiments/2
🏃 View run random-forest-classifier at: http://localhost:5000/#/experiments/2/runs/acbc3dce4cf742be985e6be4eb60c65e
🧪 View experiment at: http://localhost:5000/#/experiments/2


In [40]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
EXPERIMENT_NAME = "rf-classifier-nested-tuning"
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

df = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    output_format="pandas",
)

In [41]:
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_score,metrics.accuracy_score_X_test,metrics.training_accuracy_score,metrics.training_recall_score,...,params.n_estimators,tags.mlflow.user,tags.estimator_name,tags.estimator_class,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.parentRunId,tags.mlflow.runName,tags.stage,tags.model_family
0,1281a19ad5da488dafd672f3ae0a2e83,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:38.335000+00:00,2025-10-05 05:07:29.775000+00:00,0.877248,0.637500,0.877248,0.877248,...,200,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-5,None,None
1,20c57124c48045cfacde7123836d2d56,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:27.896000+00:00,2025-10-05 05:06:38.282000+00:00,0.924159,0.640625,0.924159,0.924159,...,400,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-4,None,None
2,baf47c08bac542d59c0556288300f040,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:18.020000+00:00,2025-10-05 05:06:27.845000+00:00,0.935887,0.646875,0.935887,0.935887,...,300,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-3,None,None
3,f0ecb842d0cd461dac00f014a5c28aec,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:06.457000+00:00,2025-10-05 05:06:17.962000+00:00,0.842846,0.631250,0.842846,0.842846,...,200,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-2,None,None
4,3827d61f38614c779acd5ad76bb250c1,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:05:57.165000+00:00,2025-10-05 05:06:06.401000+00:00,1.000000,0.659375,1.000000,1.000000,...,100,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-1,None,None
5,acbc3dce4cf742be985e6be4eb60c65e,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:05:55.003000+00:00,2025-10-05 05:07:29.828000+00:00,NaN,NaN,NaN,NaN,...,None,ivanr,None,None,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,None,random-forest-classifier,tuning,RandomForestClassifier


In [42]:
# keep only runs that actually logged accuracy
df = df[df["metrics.accuracy"].notna()]
df = df.sort_values("metrics.accuracy", ascending=False)

In [43]:
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_score,metrics.accuracy_score_X_test,metrics.training_accuracy_score,metrics.training_recall_score,...,params.n_estimators,tags.mlflow.user,tags.estimator_name,tags.estimator_class,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.parentRunId,tags.mlflow.runName,tags.stage,tags.model_family
4,3827d61f38614c779acd5ad76bb250c1,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:05:57.165000+00:00,2025-10-05 05:06:06.401000+00:00,1.000000,0.659375,1.000000,1.000000,...,100,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-1,None,None
2,baf47c08bac542d59c0556288300f040,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:18.020000+00:00,2025-10-05 05:06:27.845000+00:00,0.935887,0.646875,0.935887,0.935887,...,300,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-3,None,None
1,20c57124c48045cfacde7123836d2d56,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:27.896000+00:00,2025-10-05 05:06:38.282000+00:00,0.924159,0.640625,0.924159,0.924159,...,400,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-4,None,None
0,1281a19ad5da488dafd672f3ae0a2e83,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:38.335000+00:00,2025-10-05 05:07:29.775000+00:00,0.877248,0.637500,0.877248,0.877248,...,200,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-5,None,None
3,f0ecb842d0cd461dac00f014a5c28aec,2,FINISHED,file:C:/Users/ivanr/apps/MLOps/mna-2.2-trackin...,2025-10-05 05:06:06.457000+00:00,2025-10-05 05:06:17.962000+00:00,0.842846,0.631250,0.842846,0.842846,...,200,ivanr,RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifier,c:\Users\ivanr\apps\MLOps\mna-2.2-tracking\mlo...,LOCAL,acbc3dce4cf742be985e6be4eb60c65e,trial-2,None,None


In [44]:
best_row = df.iloc[0]
best_run_id = best_row["run_id"]
best_accuracy = best_row["metrics.accuracy"]

In [45]:
# Fetch the params of the best run (handy for auditing/promotion message)
best_run = client.get_run(best_run_id)
best_params = best_run.data.params

In [46]:
best_params

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'class_weight': 'None',
 'criterion': 'gini',
 'max_depth': 'None',
 'max_features': 'sqrt',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '100',
 'n_jobs': '-1',
 'oob_score': 'False',
 'random_state': '42',
 'verbose': '0',
 'warm_start': 'False'}

# Register Model

In [47]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run_id}/model",
    name="rf-classifier"
)
result

Successfully registered model 'rf-classifier'.
2025/10/04 23:11:57 WARNING mlflow.tracking._model_registry.fluent: Run with id 3827d61f38614c779acd5ad76bb250c1 has no artifacts at artifact path 'model', registering model based on models:/m-b929f35b8b2c46fcb89393af08d72390 instead
2025/10/04 23:11:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rf-classifier, version 1
Created version '1' of model 'rf-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1759641117266, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1759641117266, metrics=None, model_id=None, name='rf-classifier', params=None, run_id='3827d61f38614c779acd5ad76bb250c1', run_link='', source='models:/m-b929f35b8b2c46fcb89393af08d72390', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [48]:
client.set_registered_model_alias(
    name="rf-classifier",
    alias="champion",
    version=result.version  
)

In [49]:
best_run_id

'3827d61f38614c779acd5ad76bb250c1'

In [ ]:
# !mlflow models serve -m "models:/rf-classifier@champion" --host 0.0.0.0 --port 7000 --env-manager local

# Test the served model

In [50]:
X_test.iloc[0:2].values.tolist()

[[7.7, 0.56, 0.08, 2.5, 0.114, 14.0, 46.0, 0.9971, 3.24, 0.66, 9.6],
 [7.8,
  0.5,
  0.17,
  1.6,
  0.0819999999999999,
  21.0,
  102.0,
  0.996,
  3.39,
  0.48,
  9.5]]

In [51]:
import requests, json

url = "http://localhost:7000/invocations"
payload = {
    "inputs": X_test.iloc[0:2].values.tolist()
}

r = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(payload))
print(r.status_code, r.text)

200 {"predictions": [5, 5]}
